# Conversation AI: Assignment 2
## Tasks
### Component: 1. Data Collection & Preprocessing

**Download the last two years of financials (Use any one group member's company earning statements, if nothing is available use any company's data freely available). Clean and structure the data for retrieval.**

Download AAPL financials (PDF)

In [1]:
import requests

# make a dir to store the pdfs
import os
pdf_dl_path = './financial-docs-raw'
os.makedirs(pdf_dl_path, exist_ok=True)

for url in [
    'https://s2.q4cdn.com/470004039/files/doc_earnings/2023/q4/filing/_10-K-Q4-2023-As-Filed.pdf',
    'https://s2.q4cdn.com/470004039/files/doc_earnings/2024/q4/filing/10-Q4-2024-As-Filed.pdf',
    ]:
    response = requests.get(url)

    with open(f'{pdf_dl_path}/' + url.split('/')[-1], 'wb') as file:
        file.write(response.content)

Convert the text to Markdown to enable better ingestion for later retrieval

In [3]:
!pip install markitdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 880.4 kB/s eta 0:00:0031m1.2 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 1.8 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached xlrd-2.0.1-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached youtube_transcript_api-0.6.3-py3-none-any.whl.metadata (17 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached portalocker-2.10.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 979.8 kB/s eta 0:00:00B/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.2/188.2 kB 2.6 MB/s et

In [4]:
from markitdown import MarkItDown

def get_markdown_from_pdf(pdf_file_path):
    md = MarkItDown()
    result = md.convert(pdf_file_path)
    return result.text_content

md_output_path = './financial-docs-md'
os.makedirs(md_output_path, exist_ok=True)

for pdf_file in os.listdir(pdf_dl_path):
    pdf_file_path = os.path.join(pdf_dl_path, pdf_file)
    markdown_content = get_markdown_from_pdf(pdf_file_path)
    
    md_file_path = os.path.join(md_output_path, pdf_file.replace('.pdf', '.md'))
    with open(md_file_path, 'w') as md_file:
        md_file.write(markdown_content)

### Component: 2. Basic RAG Implementation

- **Convert financial documents into text chunks.**

Now that we have the markdown version of the pdfs, let's use Langchain's wonderful markdown splitter to split the files to chunks.

In [6]:
!pip install langchain
!pip install qdrant-client


[notice] A new release of pip is available: 24.1.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  Using cached qdrant_client-1.13.2-py3-none-any.whl.metadata (10 kB)
  Using cached grpcio_tools-1.70.0-cp312-cp312-macosx_10_14_universal2.whl.metadata (5.3 kB)
  Using cached protobuf-5.29.3-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached grpcio-1.70.0-cp312-cp312-macosx_10_14_universal2.whl.metadata (3.9 kB)
  Using cached hyperframe-6.1.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached hpack-4.1.0-py3-none-any.whl.metadata (4.6 kB)
Using cached qdrant_client-1.13.2-py3-none-any.whl (306 kB)
Using cached grpcio_tools-1.70.0-cp312-cp312-macosx_10_14_universal2.whl (6.0 MB)
Using cached grpcio-1.70.0-cp312-cp312-macosx_10_14_universal2.whl (11.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
Using cached protobuf-5.29.3-cp38-abi3-macosx_10_9_universal2.whl (417 kB)
Using cached hpack-4.1.0-py3-none-any.whl (34 k

In [ ]:
from langchain.text_splitter import MarkdownTextSplitter

# Initialize the markdown splitter
splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=100)

# Split the markdown files into chunks
for md_file in os.listdir(md_output_path):
    md_file_path = os.path.join(md_output_path, md_file)
    with open(md_file_path, 'r') as file:
        markdown_content = file.read()
    
    chunks = splitter.split_text(markdown_content)
    
    # Save the chunks to new files
    chunk_output_path = os.path.join(md_output_path, 'chunks')
    os.makedirs(chunk_output_path, exist_ok=True)
    
    for i, chunk in enumerate(chunks):
        chunk_file_path = os.path.join(chunk_output_path, f'{md_file.replace(".md", "")}_chunk_{i}.md')
        with open(chunk_file_path, 'w') as chunk_file:
            chunk_file.write(chunk)

/var/folders/8s/cgfskf4s08sf0wtns9byv36r0000gn/T/ipykernel_57778/3769577374.py:7: UserWarning: Api key is used with an insecure connection.
  qdrant_client = QdrantClient("http://localhost:6333", api_key='secret123')
/var/folders/8s/cgfskf4s08sf0wtns9byv36r0000gn/T/ipykernel_57778/3769577374.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


- **Embed using a pre-trained model**

In [10]:
!pip install sentence-transformers

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=57778) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached regex-2024.11.6-cp312-cp312-macosx_11_0_arm64.whl.metadata (40 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 6.0 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 11.7 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 3.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 2.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 4.4 MB/s eta 0:00:00m eta 0:00:0136m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 7.5 MB/s eta 0:00:00
Using cached regex-2024.11.6-cp312-cp312-macosx_11_0_arm64.whl (284 kB)
   ━━━━━━━━━━━━

- **Store and retrieve using a basic vector database**

In [15]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct
from sentence_transformers import SentenceTransformer
from uuid import uuid4

# Initialize Qdrant client
qdrant_client = QdrantClient("http://localhost:6333", api_key='secret123')

# Create a collection in Qdrant
collection_name = "financial_docs"
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config={"size": 384, "distance": "Cosine"}
)

# Load a pre-trained model for embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

# Embed the chunks and upsert to Qdrant collection
for chunk_file in os.listdir(os.path.join(md_output_path, 'chunks')):
    chunk_file_path = os.path.join(md_output_path, 'chunks', chunk_file)
    with open(chunk_file_path, 'r') as file:
        chunk_content = file.read()
    
    # Generate embeddings for the chunk
    embeddings = model.encode([chunk_content])
    # import pdb;pdb.set_trace()
    # Create a point structure for Qdrant
    points = [
        PointStruct(
            id=str(uuid4()),
            vector=embeddings[0],
            payload={"text": chunk_content, "filename": chunk_file}
        )
    ]
    
    # Upsert the points to the Qdrant collection
    qdrant_client.upsert(
        collection_name=collection_name,
        points=points
    )

/var/folders/8s/cgfskf4s08sf0wtns9byv36r0000gn/T/ipykernel_57778/1459721950.py:7: UserWarning: Api key is used with an insecure connection.
  qdrant_client = QdrantClient("http://localhost:6333", api_key='secret123')
/var/folders/8s/cgfskf4s08sf0wtns9byv36r0000gn/T/ipykernel_57778/1459721950.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


In [21]:
def retrieve_relevant_chunks(query, top_k=5):
    # Generate embeddings for the query
    query_embedding = model.encode([query])[0]

    # Search for the most relevant chunks in the Qdrant collection
    search_result = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=top_k
    )

    # Extract and return the relevant chunks
    relevant_chunks = [hit.payload['text'] for hit in search_result]
    return relevant_chunks

# Example usage
query = "iPad"
relevant_chunks = retrieve_relevant_chunks(query, top_k=1)
for i, chunk in enumerate(relevant_chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")

Chunk 1:
iPad

iPad net sales decreased during 2024 compared to 2023 due primarily to lower net sales of iPad Pro and the entry-level iPad
models, partially offset by higher net sales of iPad Air.

Wearables, Home and Accessories

Wearables,  Home  and  Accessories  net  sales  decreased  during  2024  compared  to  2023  due  primarily  to  lower  net  sales  of
Wearables and Accessories.

Services

Services net sales increased during 2024 compared to 2023 due primarily to higher net sales from advertising, the App Store®
and cloud services.

Apple Inc. | 2024 Form 10-K | 23

Gross Margin

Products and Services gross margin and gross margin percentage for 2024, 2023 and 2022 were as follows (dollars in millions):

Gross margin:

Products

Services

Total gross margin

Gross margin percentage:

Products

Services

Total gross margin percentage

Products Gross Margin

2024

2023

2022

$

$

109,633  $

108,803  $

114,728

71,050

60,345

56,054

180,683  $

169,148  $

170,782

 37.2

/var/folders/8s/cgfskf4s08sf0wtns9byv36r0000gn/T/ipykernel_57778/340242783.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant_client.search(
